# Google Gemini 2.0 with ADK (Agent Development Kit) and MCP (Model Context Protocol) Servers

Agent Development Kit (ADK) is a new modular framework for developing and deploying AI agents. ADK makes it easy to get started with simple agents powered by Gemini models and Google AI tools while providing the control and structure needed for more complex agent architectures and orchestration.

Gemini models can be used with MCP server using its native tool calling capabilities. MCP, or Model Context Protocol, is an open standard introduced by Anthropic designed to standardize how AI models like Gemini interact with external tools and data sources. Instead of requiring custom integrations for each tool, MCP provides a structured way for models to access context, such as functions (tools), data sources (resources), or pre-defined prompts. This allows AI agents to securely and efficiently connect with real-world systems and workflows.

MCP server expose their tools via JSON schema definitions, which can be converted to Gemini compatible OpenAPI schema definitions. This allows you to easily use MCP server with Gemini models, below you will example on how to implement this. 

You can learn more about Google Search integration with Gemini here:
- [https://ai.google.dev/gemini-api/docs/function-calling?lang=python](https://ai.google.dev/gemini-api/docs/function-calling?lang=python&example=weather)
- [https://google.github.io/adk-docs/tools/mcp-tools/](https://google.github.io/adk-docs/tools/mcp-tools/)

In [ ]:
%pip install google-adk --upgrade

In [11]:
import os
from google.genai import types
from google.adk import Agent, Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

async def mcp_agent():
        # --- Step 1: Connect to MCP Server and Get Tools ---
        tools, exit_stack = await MCPToolset.from_server(
            connection_params=StdioServerParameters(
            command="npx", 
            args=["-y", "@philschmid/weather-mcp"], 
             # connection_params=SseServerParams(url="http://remote-server:port/path", headers={...})
            )
        )

        # --- Step 2: Define the Agent ---
        root_agent = Agent(
            model="gemini-2.0-flash",
            name='weather_assistant',
            instruction='Can read the weather in a specific city',
            tools=tools, 
            
        )

        # --- Step 3: Setup Session, Runner, and Execute ---
        # Session saves the conversation history and state
        session_service = InMemorySessionService()
        session = session_service.create_session(
            state={}, app_name='weather_app', user_id='user_fs'
        )
        # Runner is responsible for executing the agent
        runner = Runner(
            app_name='weather_app',
            agent=root_agent,
            session_service=session_service,
        )

        # --- Step 4: Run the agent ---
        prompt = "What is the weather in Berlin tomorrow, 2025/04/15?"
        user_content = types.Content(role='user', parts=[types.Part(text=prompt)])

        events_async = runner.run_async(
            session_id=session.id, user_id=session.user_id, new_message=user_content
        )

        async for event in events_async:
            if event.content.parts[0].function_call:
                print(f"Function call: {event.content.parts[0].function_call.args}")
            elif event.content.parts[0].function_response:
                print(f"Function response: {event.content.parts[0].function_response.response}")
            else:
                print(event.content.parts[0].text)
   

        # --- 5: Cleanup MCP server connection ---
        if exit_stack:
            await exit_stack.aclose()

await mcp_agent()

Function call: {'location': 'Berlin, DE', 'date': '2025-04-15'}
Function response: {'result': CallToolResult(meta=None, content=[TextContent(type='text', text='{"2025-04-15T00:00":14.1,"2025-04-15T01:00":13.8,"2025-04-15T02:00":13.4,"2025-04-15T03:00":13.1,"2025-04-15T04:00":12.6,"2025-04-15T05:00":12.1,"2025-04-15T06:00":12.9,"2025-04-15T07:00":14.4,"2025-04-15T08:00":17,"2025-04-15T09:00":19.4,"2025-04-15T10:00":20.3,"2025-04-15T11:00":20.7,"2025-04-15T12:00":21.1,"2025-04-15T13:00":20.9,"2025-04-15T14:00":20.1,"2025-04-15T15:00":19.3,"2025-04-15T16:00":18.9,"2025-04-15T17:00":18.3,"2025-04-15T18:00":17.7,"2025-04-15T19:00":16.8,"2025-04-15T20:00":16.1,"2025-04-15T21:00":15.5,"2025-04-15T22:00":14.9,"2025-04-15T23:00":14.4}', annotations=None)], isError=False)}
OK. The weather in Berlin tomorrow, 2025-04-15, will be as follows: The temperature in Berlin will be between 12.1 and 21.1 degrees Celsius.
